# Entrenamiento del modelo

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

# Define las rutas a las carpetas de entrenamiento y validación
train_dir = 'directorio_datos_train'
val_dir = 'directorio_datos_val'


# Parámetros de entrenamiento
image_size = (224, 224)  
batch_size = 32
num_classes = 5

# Crea los generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Cargar el modelo ResNet101 con pesos pre-entrenados
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas para la clasificación
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Configurar checkpoint para guardar el mejor modelo
checkpoint = ModelCheckpoint('best_modell.keras', monitor='val_accuracy', save_best_only=True, mode='max')

# Entrenar el modelo
history=model.fit(train_generator, 
          epochs=12,
          validation_data=val_generator,
          callbacks=[checkpoint])


# Guarda el modelo

In [ ]:
model.save('mobilnetV2_2.keras')

# Matriz de confusión

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Ruta a la carpeta de prueba
test_dir = 'directorio_datos_test'

# Crear un generador de datos para los datos de prueba
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False) # Importante: no mezclar los datos para comparar con las etiquetas reales

# Cargar el mejor modelo
model = tf.keras.models.load_model('/kaggle/working/best_modell.keras')

# Realizar predicciones
# Realizar predicciones
predictions = model.predict(test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size)))

# Obtener la clase con la mayor probabilidad para cada imagen
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas reales
true_classes = test_generator.classes

# Calcular la matriz de confusión
cm = confusion_matrix(true_classes, predicted_classes)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual labels')
plt.xlabel('Predicted labels')
plt.savefig('matriz,jpg')
plt.show()


# Recall y Graficas de entrenamiento 

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score
import numpy as np
# Graficar precisión y pérdida
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.savefig('loss.jpg')
plt.show()

recall = recall_score(true_classes, predicted_classes, average='macro')

print(f'Recall: {recall}')